# Problem Introduction

Here we introduce the problem and talk about 2D version, bending, 1D version, and how we get to weak form and solver.

In [ ]:
%%javascript
MathJax.Hub.Config({TeX: {equationNumbers: {autoNumber: "AMS"}}});

In [ ]:
# from jupyterthemes import jtplot
# jtplot.style(theme='grade3')

In [ ]:
# %load_ext tikzmagic

# Step Index fiber geometry and parameters

The step index fiber consists of a tube of glass called the core encased in an annulus of glass with lower refractive index, known as the cladding, all encased in polymer.  It conducts light in the core by total internal reflection.  The light is studied by breaking it up into modes that solve the mode equation.  The geometry of the fiber is shown here:

In [ ]:
%%tikz
\filldraw[fill=gray!50, very thin ] (0,0) circle (1);
\draw[gray, very thin] (0,0) circle(3);
%\draw[gray, thin] (-3.5,0) -- (3.5,0);
%\draw[gray, thin] (0, -3.5) -- (0, 3.5);
\draw[gray, thin] (0,0) -- (30:1);
\draw[gray, thin] (0,0) -- (30:1);



# One dimensional Transverse Fiber

Taking a cross section above and finding its refractive index gives a one dimensional version of the problem.  The geometry and refractive index function is shown below:

In [ ]:
%%tikz
\draw[gray, thin] (-3.5, 0) -- (3.5, 0);



## Mode Equation

The modes of the fiber solve the non-dimensionalized Helmholtz equation:

$$
\begin{align}
u'' + L^2\,k^2\,\hat{n}^2\,u &= L^2\beta^2 u\label{eq:helmholtz} \quad x \in [-R_{outer}, R_{outer}]
\end{align}
$$

where $\hat{n}$ is the non-dimensionalized refractive index function, $L$ is the scaling factor, and $k$ is the wavenumber, defined as $2\pi/\lambda$ where $\lambda$ is the operating wavelength.

We transform (\ref{eq:helmholtz}) by subtracting $L^2 k^2 n_0^2 \, u$ from both sides giving

$$
\begin{align}
u'' - \big(L^2 k^2 n_0^2 - L^2 k^2 \hat{n}^2 \big)\,u & = -\big(L^2 k^2 n_0^2 - L^2 \beta^2\big)\,u.
\end{align}
$$

By defining 

$$
\begin{align}
V   & = L^2 k^2 \big( n_0^2 - n_1^2 \big)\\
Z^2 & = L^2 \big(k^2 n_0^2 - \beta^2\big)
\end{align}
$$
we can rewrite the orginal equation as

$$
\begin{align}
- u" + \,V\,\chi_{[-R_0,R_0]}\, u &= Z^2 u\label{helm2}
\end{align}
$$

where $\chi_{[-R_0,R_0]}$ is the characteristic function of $[-R_0,R_0].$ By writing the equation in this way, we see it is the eigenproblem for the one-dimensional Schroedinger operator.


## Boundary Conditions

We now turn to deciding on appropriate boundary conditions. First note that if $|x| > R_0$ then equation \ref{helm2} simplifies to $u" + Z^z u = 0,$ which is well known to have complex exponential solutions.  We thus impose boundary conditions that reflect this, namely:

$$
\begin{align}
u'(R_{outer}) - iZu(R_{outer}) & = 0 \\
u'(-R_{outer}) + iZu(-R_{outer}) & = 0.
\end{align}
$$

these together with the modified Helmholtz equation above form our problem: Given a specified function $V$ find a complex $Z$ and non-trivial $u$ satisfying

$$
u" + \,V^2\,\chi_{[-R_0,R_0]}\, u = -Z^2u \quad x\in(-R_{outer}, R_{outer})\\
u'(R_{outer}) - iZu(R_{outer})  = 0 \\
u'(-R_{outer}) + iZu(-R_{outer})  = 0.
$$


## Weak Form and Polynomial Eigenvalue Problem

For simplicity of notation, let $\hat{V}$ denote $V^2\,\chi_{[-R_0,R_0]}$ and $R$ denote $R_{outer}$. Then the weak form of our problem is to find a complex $Z$ and function $u$ such that for all test functions $v$ we have

$$
\begin{align}
\int\big( u' v' - \hat{V}\, u\, v\big)\; \mbox{d}x - u'\,v\,\big|^R_{-R} = Z^2 \int u\, v\; \mbox{d}x.
\end{align}
$$

By utilizing the boundary conditions we see that

$$  u'\,v\,\big|^R_{-R} = iZ\,u(R)\,v(R) + iZ\,u(-R)\,v(-R)$$

which overall gives the weak form

$$
\begin{align}
\int\big( u' v' - \hat{V}\, u\, v\big)\; \mbox{d}x - Z\Big[i\,u(R)\,v(R) + i\,u(-R)\,v(-R)\Big] - Z^2 \int u\, v\; \mbox{d}x = 0.
\end{align}
$$

which we recognize as a polynomial eigenvalue problem in $Z$. 

The appearence of this polynomial eigenproblem allows us to use techniques and code developed previously to find the modes and eigenvalues.  This has been implemented in the function 'modes':

# Building the 1D Transverse Fiber using Fiber1D class

We can create the above geometry and refractive index function using the fiber1D class.  We can then use methods in the class to solve for the modes.

In [ ]:
import numpy as np
from bending.one_D.fiber1D import fiber1D
from bending.one_D.fiber1D.parameters.base import params
from ngsolve import x, CoefficientFunction, grad

To create the fiber, we pass a parameter dictionary to the constructor:

In [ ]:
A = fiber1D(params)

## Geometric Parameters

The radius of the core and cladding are denoted by $r_{core}$ and $r_{clad}$ respectively.  The radius of the polymer is $r_{polymer}.$ There is also an outer region of homogeneous material that extends beyond the polymer cladding to the radius $r_{outer}$. Typically this material is air or vacuum. One can also specify a buffer region between the polymer and outer region if one likes.

We fix a length and denote this by the variable $L$ which we will use to non-dimensionalize the problem.  The scaled radii from above are denoted with capital 'R's in place of lower case 'r's.


In [ ]:
# Define dimensional radii and scale factor

r_core = 1e-6
r_clad = 17e-6
r_polymer = 17e-6
r_buffer = 17e-6
r_outer = 19e-6

L = 1e-6

# Non-Dimensionalize
Router = r_outer/L
R_core = r_core/L
R_clad = r_clad/L



## Material/Physical Parameters

Prior to bending the fiber, the material in each region has a constant refractive index, which we specify in the step function $n(x)$:

$$
\begin{equation}
n(x) = \begin{cases} 
      n_{core} & |x| < R_{core} \\
      n_{clad} & |x| \in (R_{core}, R_{clad})\\
      n_0    & |x| \in (R_{clad}, R_{outer}).
       \end{cases}
\end{equation}
$$

We generally group the geometry of interest and the refractive index function into two parts. For the geometry, we assume that all variation of material and refractive index occurs inside a ball of radius $R_0,$ and that the material outside this ball is homogeneous. This naturally divides the refractive index function into two parts, which we label $n_1(x)$ and $n_0,$ defined inside and outside the circle of radius $R_0$ respectively.  The function $n_1(x)$ specifies the refractive index of each material at each point inside the region of variation, while the constant function $n_0$ specifies the refractive index of the ambient material. Typically $n_0$ is the refractive index of air or vacuum. Thus we can write

$$
\begin{equation}
n(x) = \begin{cases} 
      n_1(x) & |x| < R_0 \\
      n_0    & |x|  >   R_0
       \end{cases}
\end{equation}
$$

where for us $R_0 = R_{clad}$ and 

$$
\begin{equation}
n_1(x) = \begin{cases} 
      n_{core} & |x| < R_{core} \\
      n_{clad} & |x| \in (R_{core}, R_{clad})\\
       \end{cases}
\end{equation}
$$

Bending applies stress to the fiber in the core and cladding regions, modifying their refractive indices.  The bent fiber is approximately circular locally, with radius $R_{bend}$.  The modified refractive index function has the form

$$
\begin{equation}
n_{material}(x) = \begin{cases} 
      n_1\big(1 - \frac{n_1^2x}{2R_{bend}}(P_{12} -\nu(P_{11} + P_{12})\big) & |x| < R_0 \\
      n_0    & |x| >   R_0
       \end{cases}
\end{equation}
$$

where $\nu$ is Poisson's ration and the $P_{ij}$ are elements of the photo-elastic tensor.  To simplify notation we define $C = P_{12} -\nu(P_{11} + P_{12}).$


In [ ]:
n0 = 1
n_core = 1.48
n_clad = 1.38

nu = 1/3
P12 = 2
P11 = 1
C = P12 - nu * (P11 + P12)  # need to enter accurate quantities above

# Define operating wavelength and wavenumber.
wl = 1.55e-6
k = np.pi/wl

Below we see the parameters contained in the dictionary.  All are named as seen above.

In [ ]:
params

The fiber starts with a standard refractive index function ($N$) and associated $V$ function, which we can easily visualize:

In [ ]:
A.plotN()

# Mode Finding

We can find the unbent modes of the waveguide by calling the 'modes' method.  Pick a center and radius in the complex pland in which to look

In [ ]:
import ngsolve as ng
from ngsolve.webgui import Draw

In [ ]:
z, y, fes, history, P = A.modes(center=4.3479j, rad=.005,p=3, nspan=3,
                                npts=4, niterations=20)

In [ ]:
from ngsolve.internal import viewoptions, visoptions

viewoptions.drawcolorbar = False
viewoptions.drawedges = 1
visoptions.autoscale = False
visoptions.deformation = True
visoptions.scaledeform1 = 0.5 

In [ ]:
for i in range(len(y)):
    A.plotsol(z[i],y[i],fes)

Then visualize the solutions like this:

# Bending

We can update the refractive index function $N$ to any ngsolve coefficient function.  This automatically updates the $V$ function as well.  To bend the fiber, we first create the bending parameters:

In [ ]:
nu = 1/3
P12 = 2
P11 = 1
C = P12 - nu * (P11 + P12)  # need to enter accurate quantities above

# Bending radius
r_bend = 6 * 100e-6    
R_bend = r_bend/A.L


Next we create the modified refractive index function:

In [ ]:
# Note: it would be great if we could do this more easily using A.N itself
n1 = CoefficientFunction([0, A.n_clad, A.n_core]) 
n0_cf = CoefficientFunction([A.n0, 0, 0])

n_material = (n1 * (1 - (n1**2 * x) / ( 2 * R_bend) * C)) + n0_cf


Now we set the fiber's $N$ function:


In [ ]:
A.N = n_material

In [ ]:
A.plotN()

In [ ]:
A.plotV()

To find the modes, we use the 'modes' method:

In [ ]:
z, y, fes, history, P = A.modes(center=4.3479j, rad=.005, p=4, nspan=3,
                                npts=4, niterations=20)

In [ ]:
for i in range(len(y)):
    A.plotsol(z[i],y[i],fes)

## Calculating Losses

We can use the object to calculate beta and the confinement loss from our eigenvalues:

In [ ]:
b = A.beta(z[0])
b

In [ ]:
verts = [vtx for vtx in A.mesh.vertices]
type(verts[0])
v0 = verts[0]
v0.__dir__()
mesh(v0.point)

In [ ]:
n_material = (n1 * (1 - (n1**2 * x) / ( 2 * R_bend) * C)) + n0_cf

N_mat = np.array([n_material(mesh(pt)) for pt in pts])

fig = plt.figure(figsize=(14,3))
plt.plot(pts, N_mat)
plt.ylim(-.6,6)

In [ ]:
V_material_squared = (L * k) ** 2 * (n_material ** 2 - n0 ** 2)

V_plot = np.array([V_material_squared(mesh(pt)) for pt in pts])

fig = plt.figure(figsize=(14,3))
plt.plot(pts, V_plot)
plt.ylim(-.5,6)

In [ ]:
z,v = modes(V_material_squared, mesh, center = 2.06938782j, rad = .01, nspan = 3)

Having retrieved the modes, we visualize them:

In [ ]:
pts = [vtx.point[0] for vtx in mesh.vertices]
fig = plt.figure(figsize=(9,6))
V = ng.H1(mesh, complex = True)
i = 0
v2 = normalize(v[i],z[i], mesh, V)
V_plot = [V_material_squared(mesh(p)) for p in pts]

fig = plt.figure(figsize=(10,6))
plt.plot(pts, v2)
plt.plot(pts, .01 * np.array(V_plot), '--')